In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'
%matplotlib inline

In [2]:
import numpy as np 
import contextily as ctx
import geopandas as gpd
import matplotlib.pyplot as plt
from imview import pltlib
import pandas as pd
from pygeotools.lib import warplib,geolib,iolib,malib,filtlib,timelib
import os,sys,glob,shutil
from velocity_proc import glac_dyn, constants, velocity_filter, velocity_timeseries

In [4]:
%cd /nobackupp11/sbhusha1/chapter2/lirung/202320218/20161106_combined_hs__20171222_combined_hs_mgm_disp_9spm_spm_ker15_corr_kernel9px_resmin_txm

/nobackupp11/sbhusha1/chapter2/lirung/202320218/20161106_combined_hs__20171222_combined_hs_mgm_disp_9spm_spm_ker15_corr_kernel9px_resmin_txm


In [10]:
vx_orig_fn = '20161106_combined_hs__20171222_combined_hs_mgm_disp_9spm_spm_ker15_corr_kernel9px_resmin_txm_vx_skiprate_1_1.tif'
vy_orig_fn = '20161106_combined_hs__20171222_combined_hs_mgm_disp_9spm_spm_ker15_corr_kernel9px_resmin_txm_vy_skiprate_1_1.tif'
error_shp  = gpd.read_file('error_shp.geojson')
vx_ds,vy_ds = [iolib.fn_getds(fn) for fn in [vx_orig_fn,vy_orig_fn]]
vx_ma,vy_ma = [iolib.ds_getma(ds) for ds in [vx_ds,vy_ds]]
extent = geolib.ds_extent(vx_ds)
fig_extent = [extent[0],extent[2],extent[1],extent[3]]

In [11]:
%matplotlib widget

In [12]:
error_shp

,fid,geometry
0,1,"POLYGON ((358985.985 3123600.481, 358949.754 3..."
1,2,"POLYGON ((358226.150 3124645.128, 358214.073 3..."
2,3,"POLYGON ((358886.686 3122625.277, 358931.639 3..."
3,4,"POLYGON ((358839.721 3122354.219, 358833.682 3..."


In [18]:
f,ax = plt.subplots()
vm_orig = np.ma.sqrt(vx_ma**2+vy_ma**2)
pltlib.iv(vm_orig,ax=ax,cmap='RdYlBu_r',clim=(0,6),extent=fig_extent)
error_shp.plot(ax=ax,facecolor='none',edgecolor='white',linewidth=2)
xlim = (356814.64580645156, 359450.71339477727)
ylim = (3121935.022988668, 3126166.6051699277)
ax.set_xlim(xlim)
ax.set_ylim(ylim)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/nobackupp16/swbuild3/sbhusha1/pip_git_sw/imview/imview/lib/pltlib.py:155: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("RdYlBu_r").copy()
  cmap.set_bad('k', alpha=1)


(3121935.022988668, 3126166.6051699277)

In [17]:
xlim = (356814.64580645156, 359450.71339477727)
ylim = (3121935.022988668, 3126166.6051699277)
ylim

(3121935.022988668, 3126166.6051699277)

In [20]:
f,ax = plt.subplots()
vm_orig = np.ma.sqrt(vx_ma**2+vy_ma**2)
pltlib.iv(velocity_timeseries.mask_by_shp(error_shp.geometry,vm_orig,vx_ds,invert=True),ax=ax,cmap='RdYlBu_r',clim=(0,6),extent=fig_extent)
error_shp.plot(ax=ax,facecolor='none',edgecolor='white',linewidth=2)
xlim = (356814.64580645156, 359450.71339477727)
ylim = (3121935.022988668, 3126166.6051699277)
ax.set_xlim(xlim)
ax.set_ylim(ylim)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/nobackupp16/swbuild3/sbhusha1/pip_git_sw/imview/imview/lib/pltlib.py:155: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("RdYlBu_r").copy()
  cmap.set_bad('k', alpha=1)


(3121935.022988668, 3126166.6051699277)

In [29]:
vx_ds

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x2aaca7ced690> >

In [30]:
vx_masked = velocity_timeseries.mask_by_shp(error_shp.geometry,vx_ma,vx_ds,invert=True)
vy_masked = velocity_timeseries.mask_by_shp(error_shp.geometry,vy_ma,vy_ds,invert=True)
vx_mask_out = os.path.splitext(vx_orig_fn)[0]+'_masked.tif'
vy_mask_out = os.path.splitext(vy_orig_fn)[0]+'_masked.tif'
iolib.writeGTiff(vx_masked,vx_mask_out,src_ds=vx_ds)
iolib.writeGTiff(vy_masked,vy_mask_out,src_ds=vy_ds)

In [33]:
vx_inpaint = velocity_filter.fillgap_wrapper(vx_mask_out,method='cubic',erode_island=False,return_ma=True)
vy_inpaint = velocity_filter.fillgap_wrapper(vy_mask_out,method='cubic',erode_island=False,return_ma=True)

writing filled array at 20161106_combined_hs__20171222_combined_hs_mgm_disp_9spm_spm_ker15_corr_kernel9px_resmin_txm_vx_skiprate_1_1_masked_cubic_limit_100px.tif
writing filled array at 20161106_combined_hs__20171222_combined_hs_mgm_disp_9spm_spm_ker15_corr_kernel9px_resmin_txm_vy_skiprate_1_1_masked_cubic_limit_100px.tif


In [32]:
f,ax = plt.subplots()
vm_inpaint = np.ma.sqrt(vx_inpaint**2+vy_inpaint**2)
pltlib.iv(vm_inpaint,ax=ax,cmap='RdYlBu_r',clim=(0,6),extent=fig_extent)
error_shp.plot(ax=ax,facecolor='none',edgecolor='white',linewidth=2)
xlim = (356814.64580645156, 359450.71339477727)
ylim = (3121935.022988668, 3126166.6051699277)
ax.set_xlim(xlim)
ax.set_ylim(ylim)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/nobackupp16/swbuild3/sbhusha1/pip_git_sw/imview/imview/lib/pltlib.py:155: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("RdYlBu_r").copy()
  cmap.set_bad('k', alpha=1)


(3121935.022988668, 3126166.6051699277)

In [34]:
f,ax = plt.subplots()
vm_inpaint = np.ma.sqrt(vx_inpaint**2+vy_inpaint**2)
pltlib.iv(vm_inpaint,ax=ax,cmap='RdYlBu_r',clim=(0,6),extent=fig_extent)
error_shp.plot(ax=ax,facecolor='none',edgecolor='white',linewidth=2)
xlim = (356814.64580645156, 359450.71339477727)
ylim = (3121935.022988668, 3126166.6051699277)
ax.set_xlim(xlim)
ax.set_ylim(ylim)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/nobackupp16/swbuild3/sbhusha1/pip_git_sw/imview/imview/lib/pltlib.py:155: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("RdYlBu_r").copy()
  cmap.set_bad('k', alpha=1)


(3121935.022988668, 3126166.6051699277)

In [37]:
f,ax = plt.subplots()
vm_inpaint = filtlib.gaussfill(np.ma.sqrt(vx_masked**2+vy_masked**2),size=51)
pltlib.iv(vm_inpaint,ax=ax,cmap='RdYlBu_r',clim=(0,6),extent=fig_extent)
error_shp.plot(ax=ax,facecolor='none',edgecolor='white',linewidth=2)
xlim = (356814.64580645156, 359450.71339477727)
ylim = (3121935.022988668, 3126166.6051699277)
ax.set_xlim(xlim)
ax.set_ylim(ylim)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Applying gaussian smoothing filter with size 51 and sigma 8.333 (sum 0.996)


/nobackupp16/swbuild3/sbhusha1/pip_git_sw/imview/imview/lib/pltlib.py:155: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("RdYlBu_r").copy()
  cmap.set_bad('k', alpha=1)


(3121935.022988668, 3126166.6051699277)

In [38]:
f,ax = plt.subplots()
pltlib.iv(vm_inpaint - np.ma.sqrt(vx_masked**2+vy_masked**2),ax=ax,cmap='RdBu',clim=(-2,2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/nobackupp16/swbuild3/sbhusha1/pip_git_sw/imview/imview/lib/pltlib.py:155: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("RdBu").copy()
  cmap.set_bad('k', alpha=1)


<AxesSubplot:>

In [39]:
vx_gauss_fill = filtlib.gaussfill(vx_masked,size=51)
vy_gauss_fill = filtlib.gaussfill(vy_masked,size=51)

Applying gaussian smoothing filter with size 51 and sigma 8.333 (sum 0.996)


Applying gaussian smoothing filter with size 51 and sigma 8.333 (sum 0.996)


In [41]:
f,ax = plt.subplots()
vm_gauss = np.ma.sqrt(vx_gauss_fill**2+vy_gauss_fill**2)
pltlib.iv(vm_gauss,ax=ax,cmap='RdYlBu_r',clim=(0,6),extent=fig_extent)
error_shp.plot(ax=ax,facecolor='none',edgecolor='white',linewidth=2)
xlim = (356814.64580645156, 359450.71339477727)
ylim = (3121935.022988668, 3126166.6051699277)
ax.set_xlim(xlim)
ax.set_ylim(ylim)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/nobackupp16/swbuild3/sbhusha1/pip_git_sw/imview/imview/lib/pltlib.py:155: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("RdYlBu_r").copy()
  cmap.set_bad('k', alpha=1)


(3121935.022988668, 3126166.6051699277)

In [42]:
vx_out = os.path.splitext(vx_orig_fn)[0]+'_masked_gauss_fill51px.tif'
vy_out = os.path.splitext(vy_orig_fn)[0]+'_masked_gauss_fill51px.tif'
iolib.writeGTiff(vx_gauss_fill,vx_out,src_ds=vx_ds)
iolib.writeGTiff(vy_gauss_fill,vy_out,src_ds=vy_ds)